<a href="https://colab.research.google.com/github/Gatwaza/TorchIt/blob/main/TorchIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from keras.applications import MobileNetV2 #Pre-trained deep learning model for image classification.
from tensorflow.keras.optimizers import Adam #an optimization algorithm for weights update
from tensorflow.keras.models import Sequential #represents a linear stack of layers.
from tensorflow.keras.utils import to_categorical #convert integer class labels into one-hot encoded vectors.
from sklearn.model_selection import train_test_split #split data into training and testing sets for model evaluation.
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout #connected layer, flatten input into 1D array, normalizes the activations of previous layer and applies dropout regularization to the inputs.
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img #image preprocessing

In [ ]:
main_dir = "/kaggle/input/human-detection-dataset/human detection dataset"


In [ ]:
# Load and preprocess your dataset
def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path, 1)
    image = cv2.resize(image, (224, 224))  # Resize to the input size of MobileNetV2
    return image

In [ ]:
data = []
labels = []

for class_name in os.listdir(main_dir):
    class_dir = os.path.join(main_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = read_and_preprocess_image(image_path)
        data.append(image)
        labels.append(int(class_name))  # Assuming class names are 0 and 1

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=2)

In [ ]:
len(data[0])

224

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

736
185
736
185


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Model architecture
model = Sequential()
base_model = MobileNetV2(input_shape=(224, 224, 3))
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model with data augmentation
batch_size = 32
epochs = 20

history = model.fit(datagen.flow(np.array(X_train), y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=epochs,
                    validation_data=(np.array(X_test), y_test))

Epoch 1/20
23/23 [==============================] - 155s 6s/step - loss: 0.4488 - accuracy: 0.8125 - val_loss: 0.7287 - val_accuracy: 0.5676
Epoch 2/20
23/23 [==============================] - 120s 5s/step - loss: 0.3904 - accuracy: 0.8370 - val_loss: 0.7404 - val_accuracy: 0.5676
Epoch 3/20
23/23 [==============================] - 121s 5s/step - loss: 0.3108 - accuracy: 0.8723 - val_loss: 0.7829 - val_accuracy: 0.5676
Epoch 4/20
23/23 [==============================] - 118s 5s/step - loss: 0.3244 - accuracy: 0.8560 - val_loss: 0.7472 - val_accuracy: 0.5676
Epoch 5/20
23/23 [==============================] - 126s 5s/step - loss: 0.3012 - accuracy: 0.8832 - val_loss: 0.9135 - val_accuracy: 0.5892
Epoch 6/20
23/23 [==============================] - 121s 5s/step - loss: 0.2443 - accuracy: 0.9226 - val_loss: 0.8972 - val_accuracy: 0.5676
Epoch 7/20
23/23 [==============================] - 126s 5s/step - loss: 0.2364 - accuracy: 0.9062 - val_loss: 1.0501 - val_accuracy: 0.5676
Epoch 8/20
23

In [ ]:
# Import TensorFlow
import tensorflow as tf

# Save the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('TorchIt.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import tensorflow as tf
# Load the TFLite model
tflite_model_path = 'TorchIt.tflite'
tflite_model = tf.lite.Interpreter(model_path=tflite_model_path)

# Convert TFLite to TensorFlow SavedModel
saved_model_path = 'TorchIt'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.experimental_new_converter = True  # Use the new TFLite converter

# Convert and save the model
tf_saved_model = converter.convert()
with tf.io.gfile.GFile('TorchIt/saved_model.pb', 'wb') as f:
    f.write(tf_saved_model)


In [ ]:
!ls


sample_data  TorchIt  TorchIt.tflite
